This exercise will require you to pull some data from https://data.nasdaq.com/ (formerly Quandl API).

As a first step, you will need to register a free account on the https://data.nasdaq.com/ website.

After you register, you will be provided with a unique API key, that you should store:

Note: Use a .env file and put your key in there and python-dotenv to access it in this notebook.

The code below uses a key that was used when generating this project but has since been deleted. Never submit your keys to source control. There is a .env-example file in this repository to illusrtate what you need. Copy that to a file called .env and use your own api key in that .env file. Make sure you also have a .gitignore file with a line for .env added to it.

The standard Python gitignore is here you can just copy that.

In [1]:
# get api key from your .env file
import os
from dotenv import load_dotenv

load_dotenv()
API_KEY = os.getenv('NASDAQ_API_KEY')

# print(API_KEY)

Nasdaq Data has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Nasdaq Data API instructions here: https://docs.data.nasdaq.com/docs/in-depth-usage

While there is a dedicated Python package for connecting to the Nasdaq API, we would prefer that you use the requests package, which can be easily downloaded using pip or conda. You can find the documentation for the package here: http://docs.python-requests.org/en/master/

Finally, apart from the requests package, you are encouraged to not use any third party Python packages, such as pandas, and instead focus on what's available in the Python Standard Library (the collections module might come in handy: https://pymotw.com/3/collections/). Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists. You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [2]:
# First, import the relevant modules
import requests
import json
from datetime import datetime, date

Note: API's can change a bit with each version, for this exercise it is reccomended to use the nasdaq api at https://data.nasdaq.com/api/v3/. This is the same api as what used to be quandl so https://www.quandl.com/api/v3/ should work too.

Hint: We are looking for the AFX_X data on the datasets/FSE/ dataset.

In [3]:
# Now, call the Nasdaq API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned

# Specify url
url = 'https://data.nasdaq.com/api/v3/datasets/FSE/AFX_X'

# Package the request
r = requests.get(url, API_KEY)

# Decode the data into a JSON dict
json_data = r.json()

In [4]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure

# Show the dict
json_data['dataset'];

In [5]:
# Output of this is too big. There are too many 'data' values, let's shorten it (to preview)
json_no_data_preview =  json_data['dataset'].copy()

# Preview the dict without 'data'
json_no_data_preview['data'] = "_____DATA SITS HERE_____"
json_no_data_preview

{'id': 10095370,
 'dataset_code': 'AFX_X',
 'database_code': 'FSE',
 'name': 'Carl Zeiss Meditec (AFX_X)',
 'description': 'Stock Prices for Carl Zeiss Meditec (2020-11-02) from the Frankfurt Stock Exchange.<br><br>Trading System: Xetra<br><br>ISIN: DE0005313704',
 'refreshed_at': '2020-12-01T14:48:09.907Z',
 'newest_available_date': '2020-12-01',
 'oldest_available_date': '2000-06-07',
 'column_names': ['Date',
  'Open',
  'High',
  'Low',
  'Close',
  'Change',
  'Traded Volume',
  'Turnover',
  'Last Price of the Day',
  'Daily Traded Units',
  'Daily Turnover'],
 'frequency': 'daily',
 'type': 'Time Series',
 'premium': False,
 'limit': None,
 'transform': None,
 'column_index': None,
 'start_date': '2000-06-07',
 'end_date': '2020-12-01',
 'data': '_____DATA SITS HERE_____',
 'collapse': None,
 'order': None,
 'database_id': 6129}

# These are your tasks for this mini project:
- [x] Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
- [x] Convert the returned JSON object into a Python dictionary.
- [x] Calculate what the highest and lowest opening prices were for the stock in this period.
- [x] What was the largest change in any one day (based on High and Low price)?
- [x] What was the largest change between any two days (based on Closing Price)?
- [x] What was the average daily trading volume during this year?
- [x] (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

## Collect data for the whole year 2017

In [6]:
# Getting the column names as list
column_names = json_data['dataset']['column_names']

# Getting all data for the year 2017
data_2017 = [record for record in json_data['dataset']['data'] if record[0].startswith('2017')]

## Convert the returned JSON object into a Python dictionary

In [7]:
# Creating initial dictionary
dict_2017 = {key: [value[i] for value in data_2017] for i, key in enumerate(column_names)}
dict_2017.keys()

# Convert 'Date' string values to date type
date_values = [datetime.strptime(date_str, '%Y-%m-%d').date() for date_str in dict_2017['Date']]
# date_values = [datetime.date() for datetime in datetime_values]

# Creating working dictionary indexed by date
dict_2017_by_date = {
    date: {
        'Open': open_val,
        'High': high_val,
        'Low': low_val,
        'Close': close_val,
        'Change': change_val,
        'Traded Volume': traded_val,
        'Turnover': turnover_val,
        'Last Price of the Day': last_price_val,
        'Daily Traded Units': daily_traded_val,
        'Daily Turnover': daily_turnover_val
    } for date,
    open_val,
    high_val,
    low_val,
    close_val,
    change_val,
    traded_val,
    turnover_val,
    last_price_val,
    daily_traded_val,
    daily_turnover_val in zip(
        date_values,
        dict_2017['Open'],
        dict_2017['High'],
        dict_2017['Low'],
        dict_2017['Close'],
        dict_2017['Change'],
        dict_2017['Traded Volume'],
        dict_2017['Turnover'],
        dict_2017['Last Price of the Day'],
        dict_2017['Daily Traded Units'],
        dict_2017['Daily Turnover'],
    )
}

# Sorting dates in ascending order
dict_2017_by_date = dict(sorted(dict_2017_by_date.items()))

# Printing the first date values
print(f'Values for {list(dict_2017_by_date.keys())[0].strftime("%Y-%m-%d")}:')
dict_2017_by_date[list(dict_2017_by_date.keys())[0]]

Values for 2017-01-02:


{'Open': 34.99,
 'High': 35.94,
 'Low': 34.99,
 'Close': 35.8,
 'Change': None,
 'Traded Volume': 44700.0,
 'Turnover': 1590561.0,
 'Last Price of the Day': None,
 'Daily Traded Units': None,
 'Daily Turnover': None}

## Calculate what the highest and lowest opening prices were for the stock in this period.

In [8]:
# Function to calculate min and max values of a list omitting None
def get_min_max(values_list:list):
    '''Calculates min and max values of a list provided: (min, max)'''
    min_value = min((value for value in values_list if value is not None), default=None)
    max_value = max((value for value in values_list if value is not None), default=None)
    print(f'min: {min_value}\nmax: {max_value}\n')
    return min_value, max_value

# Function to get the list of prices for set stock feature for a time series dictionary
def get_values_list(dictionary, stk_stat):
    price_list = []
    for date in list(dictionary.keys()):
        price_list.append(dictionary[date][stk_stat])
    return price_list

# Defining the stock exchange status (feature) we are interested in
stock_status = 'Open'

# Getting list of all prices for set stock_status (Opening prices)
open_price_list = get_values_list(dict_2017_by_date, stock_status)

# Getting and printing the output
print('The highest and lowest opening prices were for the stock in 2017')
get_min_max(open_price_list)

The highest and lowest opening prices were for the stock in 2017
min: 34.0
max: 53.11



(34.0, 53.11)

## What was the largest change in any one day (based on High and Low price)?

In [9]:
# Creating function to fill the dictionary with Change values
def fill_change(dictionary):
    # Filling in the 'Change' values based on 'High' and 'Low' prices, omitting rows with missing values
    for date in list(dict_2017_by_date.keys()):
        high = dict_2017_by_date[date]['High']
        low = dict_2017_by_date[date]['Low']
        if high is None or low is None:
            pass
        else:
            change = dict_2017_by_date[date]['High'] - dict_2017_by_date[date]['Low']
            dict_2017_by_date[date]['Change'] = change


# Executing function to put Change values into a dictionary
fill_change(dict_2017_by_date)
dict_2017_by_date[list(dict_2017_by_date.keys())[0]]['Change']

# Calling a function to get a list of all 'Change values'
change_list = get_values_list(dict_2017_by_date, 'Change')

# Getting the largest change in one day
print(f'The largest change in any one day based on High and Low price:\n{get_min_max(change_list)[1]}')


min: 0.18999999999999773
max: 2.8100000000000023

The largest change in any one day based on High and Low price:
2.8100000000000023


## What was the largest change between any two days (based on Closing Price)?

In [10]:
# Defining the function to calculate difference between values of two consecutive days
def get_diff_list(values_list):
    # Calculate differences between neighboring values
    return [values_list[i + 1] - values_list[i] for i in range(len(values_list) - 1)]

# Getting the list of Close values
close_list = get_values_list(dict_2017_by_date, 'Close')

# Calling a function to get the day to day change of closing price
daily_close_diff_list = get_diff_list(close_list)

# Calling a function to get the largest value and printing the result
print(f'The largest change between two days based on Closing price:\n{get_min_max(daily_close_diff_list)[1]}')

min: -2.559999999999995
max: 1.7199999999999989

The largest change between two days based on Closing price:
1.7199999999999989


## What was the average daily trading volume during this year?

In [11]:
# Creating a function to calculate the average of a list
def get_list_mean(values_list):
    return sum(values_list) / len(values_list)

# Getting the list of daily trading volume values
daily_volume_list = get_values_list(dict_2017_by_date, 'Traded Volume')

# Calling a function to get the average daily trading volume and printing the result
print(f'The average Daily Trading Volume:\n{get_list_mean(daily_volume_list)}')

The average Daily Trading Volume:
89124.33725490196


## (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [12]:
def get_list_median(values_list):
    # Sorting list in ascending order
    list_asc = sorted(values_list)

    # Getting the length of the list
    n = len(list_asc)

    # Returning the result based on even or odd number of items
    if n % 2 == 1:
        return list_asc[n // 2]
    else:
        left = list_asc[(n // 2) - 1]
        right = list_asc[n // 2]
        return (left + right) / 2

# Calling a function to get the median daily trading volume and printing the result
print(f'The average Daily Trading Volume:\n{get_list_median(daily_volume_list)}')

The average Daily Trading Volume:
76286.0
